In [ ]:
########################### Workload 1 ###############################

## https://www.codenong.com/cs106232534/
## https://www.huaweicloud.com/articles/8bf05daa2c052004c67b94aa4f071f87.html
## https://zhuanlan.zhihu.com/p/68910426
## https://spark.apache.org/docs/2.3.1/api/python/pyspark.html

In [12]:
from pyspark import SparkConf, SparkContext
spark_conf = SparkConf().setAppName("5349_a1")
sc=SparkContext.getOrCreate(spark_conf) 
text_file = sc.textFile("ALLVideos.csv")

In [17]:
from datetime import datetime
def extractcolumn(record):
    
    video_id,trending_date,category,views,likes,dislikes,country = record.split(',')
    y, d, m = trending_date.split(".")
    trending_date = datetime(int(y), int(m),int(d))
    return ((video_id,country), (trending_date, likes, dislikes))

def findMaxDate(a, b):
    if a[0] >= b[0]:
        return a
    else:
        return b

def findMinDate(a, b):
    if a[0] <= b[0]:
        return a
    else:
        return b

def difference(value):
    if value[0] == value[1]:
        dif = 0
    else:
        dif = int(value[0][2]) - int(value[1][2]) - (int(value[0][1]) - int(value[1][1]))
    return dif

def sortByValues(value):
    return value[1]


def rearrange(content):
    x, y, z = content[0][0], content[0][1], content[1]
    return (x, z, y)

In [20]:
## https://www.jianshu.com/p/af175e66ce99
## a: finding the earliest date of each category (reduceByKey)
## b: finding the latest date of each category (reduceByKey)
## d: (the latest date's dislike - the earliest date's dislike) - (the latest date's like - the earliest date's like)
## e: Reverse order
## f: take(10)

a = text_file.map(extractcolumn) \
             .reduceByKey(findMaxDate)

b = text_file.map(extractcolumn) \
             .reduceByKey(findMinDate)

c = a.join(b)
d = c.mapValues(difference)
e = d.sortBy(sortByValues,False)
f = e.map(rearrange).take(10)

In [21]:
f

[('QwZT7T-TXT0', 579119, 'GB'),
 ('QwZT7T-TXT0', 478100, 'US'),
 ('BEePFpC9qG8', 365862, 'DE'),
 ('RmZ3DPJQo2k', 334390, 'KR'),
 ('q8v9MvManKE', 299044, 'IN'),
 ('pOHQdIDds6s', 160365, 'CA'),
 ('ZGEoqPpJQLE', 151913, 'RU'),
 ('84LBjXaeKk4', 134836, 'FR'),
 ('84LBjXaeKk4', 134834, 'DE'),
 ('84LBjXaeKk4', 121240, 'RU')]

In [ ]:
########################### Workload 2 ###############################

In [14]:
import numpy as np
def splitdata(record):
    video_id,trending_date,category,views,likes,dislikes,country = record.split(',')
    return ((video_id, category), country)

def countCountry(value):
    return len(np.unique(list(value)))

def reArrangeRDD(content):
    video_id, category, count = content[0][0], content[0][1], content[1]
    return (category, count)

def reArrangeRDD_2(content):
    video_id, category, count = content[0][0], content[0][1], content[1]
    return (category, video_id)

def countID(value):
    return len(np.unique(list(value)))

def divideNum(value):
    return value[0]/value[1]

In [15]:
## z: calculating the unique contry for each key (video_id, category)
## x: counting all num for each key (category)
## y: counting all unique video_id for each key (category)
## p: combining and dividing (x.value/y.value)

z = text_file.map(splitdata).groupByKey().mapValues(countCountry)
x = z.map(reArrangeRDD).reduceByKey(lambda a,b:a+b)
y = z.map(reArrangeRDD_2).groupByKey().mapValues(countID)

p = x.join(y).mapValues(divideNum).sortBy(lambda a:a[1]).collect()

In [16]:
p

[('Trailers', 1.0),
 ('Autos & Vehicles', 1.0190448285965426),
 ('News & Politics', 1.0527098256521152),
 ('Nonprofits & Activism', 1.057344064386318),
 ('Education', 1.0628976994615762),
 ('People & Blogs', 1.063884131133748),
 ('Pets & Animals', 1.0703560703560704),
 ('Howto & Style', 1.0875230863944183),
 ('Travel & Events', 1.0929411764705883),
 ('Gaming', 1.09443748882132),
 ('Sports', 1.1421507122296848),
 ('Entertainment', 1.1446024282935856),
 ('Science & Technology', 1.1626835588828102),
 ('Film & Animation', 1.1677314564158094),
 ('Comedy', 1.2142258635136394),
 ('Movies', 1.25),
 ('Music', 1.3105183216252136),
 ('Shows', 1.555045871559633)]

In [76]:
import numpy as np
def splitCountry(record):
    video_id,trending_date,category,views,likes,dislikes,country = record.split(',')
    return (video_id,country)

def splitCategory(record):
    video_id,trending_date,category,views,likes,dislikes,country = record.split(',')
    return (video_id, category)

def listValue(value):
    return list(value)

def countCountry(content):
    video_id, category, country = content[0], np.unique(content[1][0]), content[1][1]
    countCountry = len(np.unique(list(country)))
    return [((video_id, i), countCountry) for i in category]
    
def reArrangeRDD(content):
    video_id, category, count = content[0][0], content[0][1], content[1]
    return (category, count)

def reArrangeRDD_2(content):
    video_id, category, count = content[0][0], content[0][1], content[1]
    return (category, video_id)

def countID(value):
    return len(np.unique(list(value)))

def divideNum(value):
    return value[0]/value[1]

In [81]:
## z: Grouping all country for each video_id    (video_id, list(country))
## x: Grouping all category for each video_id    (video_id, list(category))
## y: Double count  (video_id, (list(category),list(country))) --> (video_id, category, num(country)))
## p: counting all country_num for each key (category)
## q: counting all unique video_id for each key (category)
## t: combining and dividing (x.value/y.value)

z = text_file.map(splitCountry).groupByKey().mapValues(listValue)
x = text_file.map(splitCategory).groupByKey().mapValues(listValue)
y = x.join(z).flatMap(countCountry)
p = y.map(reArrangeRDD).reduceByKey(lambda a,b:a+b)
q = y.map(reArrangeRDD_2).groupByKey().mapValues(countID)
t = p.join(q).mapValues(divideNum).sortBy(lambda a:a[1]).collect()

In [83]:
t

[('Trailers', 1.0),
 ('Autos & Vehicles', 1.0190448285965426),
 ('News & Politics', 1.052844979051223),
 ('Nonprofits & Activism', 1.057344064386318),
 ('Education', 1.0628976994615762),
 ('People & Blogs', 1.0640343760329336),
 ('Pets & Animals', 1.0707850707850708),
 ('Howto & Style', 1.0876256925918326),
 ('Travel & Events', 1.0929411764705883),
 ('Gaming', 1.0946163477016635),
 ('Sports', 1.1422245184146431),
 ('Entertainment', 1.1447534885477444),
 ('Science & Technology', 1.1626835588828102),
 ('Film & Animation', 1.1677314564158094),
 ('Comedy', 1.2144120659156503),
 ('Movies', 1.25),
 ('Music', 1.310898044427568),
 ('Shows', 1.614678899082569)]